In [ ]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pickle
from flask import Flask, request, jsonify, send_file
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [3]:
dataset= "D:/Dataset_Siswa.csv"
df = pd.read_csv(dataset)

In [4]:
# Display the first few rows of the DataFrame
print(df.head())

     ID       Label  Nilai_Matematika  Nilai_Biologi  Nilai_Fisika  \
0  M001  Matematika                86             67            74   
1  M002  Matematika                81             67            77   
2  M003  Matematika                85             68            72   
3  M004  Matematika                82             65            76   
4  M005  Matematika                82             65            74   

   Nilai_Kimia   Nilai_English  Kognitif_Skills  Creativity_Innovation  \
0            75             74               99                     70   
1            70             72               94                     75   
2            71             78               93                     70   
3            73             73               87                     79   
4            69             72               93                     74   

   Communication_Collaboration  Adaptation_DecisionMaking  \
0                           73                         76   
1           

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   ID                             5000 non-null   object
 1   Label                          5000 non-null   object
 2   Nilai_Matematika               5000 non-null   int64 
 3   Nilai_Biologi                  5000 non-null   int64 
 4   Nilai_Fisika                   5000 non-null   int64 
 5   Nilai_Kimia                    5000 non-null   int64 
 6   Nilai_English                  5000 non-null   int64 
 7   Kognitif_Skills                5000 non-null   int64 
 8   Creativity_Innovation          5000 non-null   int64 
 9   Communication_Collaboration    5000 non-null   int64 
 10  Adaptation_DecisionMaking      5000 non-null   int64 
 11  Tech_Coding_Interest           5000 non-null   int64 
 12  Research_Data_Interest         5000 non-null   int64 
 13  Pro

In [6]:
# Separate features (X) and labels (y)
X = df.drop(columns=['ID', 'Label'], axis=1).values  # Drop ID and Label columns
y = df['Label'].values  # The target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Reshape the input data for LSTM
timesteps = 1
X_train = X_train.reshape(X_train.shape[0], timesteps, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], timesteps, X_test.shape[1])

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Save the scaler for later use in the Flask app
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Verify that the scaler file was saved
with open('scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)
print("Scaler loaded successfully:", type(loaded_scaler))

# Convert labels to strings and encode them
y_train = y_train.astype(str)
y_test = y_test.astype(str)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Save the label encoder for use in the Flask app
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Verify that the label encoder file was saved
with open('label_encoder.pkl', 'rb') as f:
    loaded_label_encoder = pickle.load(f)
print("Label Encoder loaded successfully:", type(loaded_label_encoder))

# Convert labels to one-hot encoding
num_classes = len(label_encoder.classes_)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)


Scaler loaded successfully: <class 'sklearn.preprocessing._data.MinMaxScaler'>
Label Encoder loaded successfully: <class 'sklearn.preprocessing._label.LabelEncoder'>


In [7]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.3094 - loss: 2.1621 - val_accuracy: 0.6889 - val_loss: 1.3986
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6920 - loss: 1.1830 - val_accuracy: 0.8022 - val_loss: 0.7523
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8193 - loss: 0.6676 - val_accuracy: 0.8422 - val_loss: 0.5228
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8749 - loss: 0.4515 - val_accuracy: 0.8756 - val_loss: 0.4001
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8963 - loss: 0.3628 - val_accuracy: 0.8956 - val_loss: 0.3310
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9083 - loss: 0.3076 - val_accuracy: 0.9133 - val_loss: 0.2890
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9221 - loss: 0.2646 - val_accuracy: 0.9156 - val_loss: 0.2625
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9334 - loss: 0.2315 - val_accuracy: 0

In [7]:
# Save the trained model for later use in the Flask app
model.save('model.keras')  # Save as Keras .keras format

# Optionally save in the H5 format too
model.save('model.h5')  # Save as .h5 format

# Save the model architecture as JSON
model_json = model.to_json()  # Get model architecture in JSON format
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

# Save the entire model using pickle (if needed)
with open('model.bin', 'wb') as bin_file:
    pickle.dump(model, bin_file)  # Save as .bin

In [8]:
# Initialize Flask app
app = Flask(__name__)

# Load the trained model, scaler, and label encoder
model = tf.keras.models.load_model('model.keras')

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Route for prediction
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Check if the 'input' field is in the request data
        data = request.get_json()

        if 'input' not in data:
            return jsonify({'error': 'No input data provided'}), 400

        # Input data should be a list (e.g., [1.0, 2.0, 3.0])
        input_data = np.array(data['input'])
        if len(input_data.shape) != 1:
            return jsonify({'error': 'Input data should be a 1D list of numerical values'}), 400

        # Reshape the input data for LSTM (1, 1, n_features)
        input_data = input_data.reshape(1, 1, len(input_data))

        # Scale the input data using the saved scaler
        input_data_scaled = scaler.transform(input_data.reshape(-1, input_data.shape[-1])).reshape(input_data.shape)

        # Make the prediction using the model
        prediction = model.predict(input_data_scaled)

        # Decode the predicted label from one-hot encoding
        predicted_label = np.argmax(prediction, axis=1)
        predicted_label = label_encoder.inverse_transform(predicted_label)

        # Return the prediction as JSON response
        return jsonify({'predicted_label': predicted_label[0]})

    except Exception as e:
        # Catch all unexpected errors and return them as a response
        return jsonify({'error': f'An error occurred: {str(e)}'}), 500
if __name__ == '__main__':
    # Set debug=False for production
    app.run(debug=False, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.3.200:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:42:08] "POST /predict HTTP/1.1" 500 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 519ms/step


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:42:54] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:43:25] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:44:47] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:45:27] "POST /predict HTTP/1.1" 200 -


In [ ]:
app = Flask(__name__)
app.has_route = set()  # Initialize a set to keep track of defined routes

@app.route('/download_model/<filename>')
def download_model(filename):
    try:
        # Check if the route for the file is already defined
        if filename in app.has_route:
            return f"Error: The route for {filename} is already defined.", 400

        # Check if the file exists
        file_path = os.path.join(os.getcwd(), filename)  # Adjust the path as needed
        if os.path.exists(file_path):
            app.has_route.add(filename)  # Add the filename to the set
            return send_file(file_path, as_attachment=True)
        else:
            return f"Error: {filename} not found.", 404
    except Exception as e:
        return f"Error: {str(e)}", 500

# Route to download scaler.pkl
@app.route('/download/scaler', methods=['GET'])
def download_scaler():
    file_path = os.path.join(os.getcwd(), 'scaler.pkl')
    if os.path.exists(file_path):
        return send_file(file_path, as_attachment=True)
    else:
        return jsonify({'error': 'scaler.pkl not found'}), 404

# Route to download label_encoder.pkl
@app.route('/download/label_encoder', methods=['GET'])
def download_label_encoder():
    file_path = os.path.join(os.getcwd(), 'label_encoder.pkl')
    if os.path.exists(file_path):
        return send_file(file_path, as_attachment=True)
    else:
        return jsonify({'error': 'label_encoder.pkl not found'}), 404

if __name__ == '__main__':
    # Set debug=False for production
    app.run(debug=False, host='0.0.0.0', port=5001)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.5:5001
INFO:werkzeug:Press CTRL+C to quit


INFO:werkzeug:127.0.0.1 - - [03/Dec/2024 18:15:28] "GET /download_model/model.h5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Dec/2024 18:16:24] "GET /download_model/model.keras HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Dec/2024 18:16:31] "GET /download_model/model.keras HTTP/1.1" 400 -
